In [15]:
import json
import requests
import urllib
import os

import pandas as pd
import numpy as np

from itertools import *

In [16]:
json_file = requests.get('https://raw.githubusercontent.com/robsontavaresjr/OD_Challange/main/supplier_car.json').text

### This .json is not on the common json format. There are linebreakers "\n" instead of commas to separate the items and the content doesnt start and ends with square brackets " [ {...}, {...}, ... {...} ] " So we do the following adjust on the file.

In [17]:
the_json_file = '[' + json_file[:-1] + ']'
the_json_file = json.loads(the_json_file.replace('\n', ','))
json_df = pd.DataFrame(the_json_file)

### Having a look into the delivery format file

In [19]:
urllib.request.urlretrieve('https://github.com/robsontavaresjr/OD_Challange/blob/main/Target%20Data.xlsx?raw=true', 
                           os.path.join(os.getcwd(), "Target Data.xlxs"))

xlsx_df = pd.read_excel(os.path.join(os.getcwd(), "Target Data.xlxs"))
xlsx_df.head()

,carType,color,condition,currency,drive,city,country,make,manufacture_year,mileage,mileage_unit,model,model_variant,price_on_request,type,zip,manufacture_month,fuel_consumption_unit
0,Convertible / Roadster,White,Used,USD,LHD,Zuzwil,CH,Toyota,1983,0.0,kilometer,Celica,2800 i Supra,False,car,NaN,3.0,l_km_consumption
1,Convertible / Roadster,Other,Original Condition,USD,LHD,London,GB,Ferrari,2001,0.0,mile,550,NaN,True,car,NaN,NaN,NaN
2,Convertible / Roadster,White,Used,USD,LHD,Scotts Valley,US,Porsche,1989,0.0,mile,"911 ""G""",NaN,True,car,NaN,NaN,NaN
3,Convertible / Roadster,Blue,Used,USD,LHD,London,GB,Rolls-Royce,1961,0.0,kilometer,Silver Cloud II,NaN,False,car,NaN,NaN,NaN
4,Convertible / Roadster,Black,Original Condition,USD,LHD,Hong Kong,HK,Gumpert,2010,0.0,kilometer,Apollo,Apollo S 800hp,False,car,NaN,NaN,NaN


### Pre Processed


In [20]:
pp_df=\
json_df[['ID', 'MakeText', 'TypeName', 'TypeNameFull', 'ModelText','ModelTypeText']].drop_duplicates().merge(
    json_df[['ID', 'Attribute Names', 'Attribute Values']]
    .drop_duplicates()
    .pivot(index='ID', columns='Attribute Names', values='Attribute Values')
    .reset_index(),
           on='ID',
           how='right'
)

## Normalizing Color, Maker, Condition, CarType

In [21]:
# Normalizing for the color
color_norm={
    'anthrazit': 'Other'
    ,'beige':'Beige'
    ,'blau':'Blue'
    ,'bordeaux':'Other'
    ,'braun':'Brown'
    ,'gelb':'Yellow'
    ,'gold':'Gold'
    ,'grau':'Gray'
    ,'grün':'Green'
    ,'orange':'Orange'
    ,'rot':'Red'
    ,'schwarz':'Black'
    ,'silber':'Silver'
    ,'violett':'Purple'
    ,'weiss':'White'
    }

pp_df.BodyColorText = pp_df.BodyColorText.apply(lambda x: color_norm[x.split()[0]])

# Normalizing for the car type
car_type_norm={
    'Cabriolet':'Convertible / Roadster'
    ,'Coupé':'Coupé'
    ,'SUV / Geländewagen':'SUV'
    ,'Pick-up':'SUV'
    ,'Kompaktvan / Minivan':'Other'
    ,'Wohnkabine':'Other'
    ,'Sattelschlepper': 'Other'
    ,'Kleinwagen':'Other'
    ,'Limousine':'Saloon'
    ,'Kombi':'Station Wagon'
    }

pp_df.BodyTypeText = pp_df.BodyTypeText.map(car_type_norm)

# Normalizing for the car condition
condition_norm={
    'Occasion':'Used'
    ,'Oldtimer':'Used'
    ,'Vorführmodell':'Original Condition'
    ,'Neu':'New'
    }

pp_df.ConditionTypeText = pp_df.ConditionTypeText.map(condition_norm)

# Normalizing for the car maker
from_to_tbl=\
(
    xlsx_df[['make']]
    .assign(_lower=lambda y: y.make.apply(lambda x: x.lower() if type(x) == str else x))
    .merge(
        pp_df[['MakeText']].assign(_lower=lambda y: y.MakeText.apply(lambda x: x.lower())),
        on='_lower',
        how='right'
    )
    .drop_duplicates()
)

# Here we need to create some kind of alert / manual validation with the client to see how we deal with new entries
# and / or brand variations... howerver as standard we will title the string, unless its on exception list
make_new_entries = from_to_tbl.loc[~from_to_tbl.make.notnull()]

exception_dict = {
    
    'BMW-ALPINA':'BMW-Alpina'
    ,'FORD (USA)':'Ford'
    
    }

make_new_entries_dict=\
dict(zip(make_new_entries.MakeText, 
         make_new_entries.MakeText.apply(lambda x: x.title())))

make_new_entries_dict=\
{
    eachKey:(make_new_entries_dict[eachKey] 
             if eachKey not in exception_dict.keys() 
             else exception_dict[eachKey]) 
    for eachKey in make_new_entries_dict.keys()
}

make_matched = from_to_tbl.loc[from_to_tbl.make.notnull()]

new_entries_norm={ **dict(zip(make_matched.MakeText, make_matched.make)), 
                   **make_new_entries_dict}

pp_df.MakeText = pp_df.MakeText.map(new_entries_norm)

In [22]:
norm_df = (
    pp_df
    .rename(
        columns={
            'BodyTypeText':'carType'
            ,'BodyColorText':'color'
            ,'ConditionTypeText':'condition'
            ,'City':'city'
            ,'MakeText':'make'
            ,'FirstRegYear':'manufacture_year'
            ,'Km':'mileage'
            ,'ModelText':'model'
            ,'ModelTypeText':'model_variant'
            ,'FirstRegMonth':'manufacture_month'
            
        }
    )
)
      
norm_df=norm_df.assign(
          currency='CHF'
          ,drive='LHD'
          ,country='CH'
          ,mileage_unit='kilometer'
          ,price_on_request=np.nan
          ,type=lambda x: x.make.apply(lambda y: 'car' if y != 'Harley-Davidson' else 'motorcycle')
          ,zip=np.nan
          ,fuel_consumption_unit=lambda x: x.ConsumptionTotalText.apply(lambda y: 'l_km_consumption' 
                                                                                   if 'l/100km' in y
                                                                                   else np.nan)
      )
norm_df.mileage = norm_df.mileage.apply(lambda x: float(x))
norm_df.manufacture_year = norm_df.manufacture_year.apply(lambda x: int(x))
norm_df.manufacture_month = norm_df.manufacture_month.apply(lambda x: int(x))


norm_df = norm_df[xlsx_df.columns]

### Integration

In [23]:
inte_df = pd.concat([norm_df, xlsx_df], ignore_index=True, sort=False)

### Excel File

In [24]:
with pd.ExcelWriter(os.path.join(os.getcwd(), 'output_data.xlsx')) as writer:  
    pp_df.to_excel(writer, sheet_name='pre_processing', index=False)
    norm_df.to_excel(writer, sheet_name='normalizating', index=False)
    inte_df.to_excel(writer, sheet_name='integration', index=False)